In [1]:
import numpy as np
import gmaps
import gmaps.datasets
import os
import requests
from ipywidgets.embed import embed_minimal_html
from scipy.interpolate import Rbf
from time import sleep

## Statistics:
- Annual PM2.5 Average in India
- Annual PM2.5 Average in New Delhi
- Heat Map of PM2.5 Average in New Delhi for 2016
- Heat Map of PM2.5 Averages in India for 2016
- Most polluted and least polluted cities in India
- Most polluted and least polluted neighborhoods in New Delhi

In [2]:
f = open('id-mappings/city-ids.txt', 'r')
cities = f.readlines()
cities = map(lambda elem: elem.split(","), cities)[1:]
cities = map(lambda elem: [elem[0], elem[1], elem[2], elem[3][:-1]], cities)
cities = filter(lambda elem: elem[2].isdigit(), cities)

In [3]:
india_data = {}
for elem in cities:
    _, stateName, _, cityName = elem
    f = open('data/{}_{}.txt'.format(stateName, cityName), 'r')
    city_data = f.readlines()
    city_data = filter(lambda elem: elem != "\n", city_data)
    city_data = map(lambda elem: elem.split(","), city_data)
    city_data = filter(lambda elem: elem[0] == "2016", city_data)
    city_data = map(lambda elem: float(elem[1].rstrip("\n")), city_data)
    if len(city_data) > 0:
        print city_data
        india_data[(stateName, cityName)] = np.mean(city_data)


[89.23, 47.35, 73.79]
[95.8]
[103.93]
[166.36]
[151.13]
[122.68, 206.29, 164.33]
[45.88]
[32.37]
[196.22]
[148.85]
[144.68, 326.36, 139.61, 278.77, 145.87, 135.92, 216.32, 119.59, 143.32, 176.39]
[124.34]
[161.04]
[53.98]
[109.92]
[183.35]
[116.41, 62.8, 45.21]
[87.03]
[64.26]
[64.14]
[39.48, 41.27]
[38.83]
[68.63]
[84.39]
[79.18, 48.61]
[49.7, 87.82, 47.47]


In [4]:
india_data = [[k, v] for (k, v) in india_data.iteritems()]
india_data = sorted(india_data, key=lambda x: x[1])

In [5]:
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [6]:
def decode_address_to_coordinates(address):
        params = {
                'address' : address,
                'sensor' : 'false',
        }  
        url = 'http://maps.google.com/maps/api/geocode/json'
        r = requests.get(url, params = params)
        return r.json()['results'][0]['geometry']['location']

In [8]:
locations = []
for (state, city), val in india_data:
    locations.append([(state, city), decode_address_to_coordinates("{}, {}".format(city, state)).values()])

In [9]:
india_coordinates = decode_address_to_coordinates("India").values()
fig = gmaps.figure(center=india_coordinates, zoom_level=4)

weights = map(lambda x: x[1], india_data)
coordinates = map(lambda x: x[1], locations)
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=weights)
heatmap_layer.max_intensity = 200
heatmap_layer.point_radius = 2.0
heatmap_layer.dissipating = False
fig.add_layer(heatmap_layer)

info_box_template = """
<div>
<p><b>City:</b> {0}, {1}</p>
<p><b>PM2.5:</b> {2:.2f}</p>
</div>
"""
city_info = [info_box_template.format(city_data[0][1], city_data[0][0], city_data[1]) for city_data in india_data]

marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)

fig

A Jupyter Widget

In [10]:
embed_minimal_html('national-aq.html', views=[fig])

In [11]:
for d in india_data[:5]:
    print "{0}: {1:.2f} ug/m3".format(d[0][1], d[1])
for d in india_data[-5:]:
    print "{0}: {1:.2f} ug/m3".format(d[0][1], d[1])

Tirupati: 32.37 ug/m3
Solapur: 38.83 ug/m3
Mumbai: 40.38 ug/m3
Visakhapatnam: 45.88 ug/m3
Panchkula: 53.98 ug/m3
Lucknow: 164.43 ug/m3
Kanpur: 166.36 ug/m3
Delhi: 182.68 ug/m3
Ahmedabad: 183.35 ug/m3
Gaya: 196.22 ug/m3


In [13]:
coordinates = np.array(coordinates)

In [14]:
x = coordinates[:, 0]
y = coordinates[:, 1]

In [15]:
# Interpolating and plotting again
rbfi = Rbf(x, y, weights, function = "inverse")

In [16]:
# sleep(0.05)
data = open('indian-cities.csv', 'r').readlines()
data = map(lambda x: x.split("\r"), data)[0]

In [17]:
cities = []
for city in data:
    cities.append(decode_address_to_coordinates("{}, India".format(city)).values())
    sleep(1)

In [18]:
cities = np.array(cities)
print cities

[[ 19.0759837  72.8776559]
 [ 28.7040592  77.1024902]
 [ 12.9715987  77.5945627]
 [ 23.022505   72.5713621]
 [ 17.385044   78.486671 ]
 [ 13.0826802  80.2707184]
 [ 22.572646   88.363895 ]
 [ 18.5204303  73.8567437]
 [ 26.9124336  75.7872709]
 [ 21.1702401  72.8310607]
 [ 26.8466937  80.946166 ]
 [ 26.449923   80.3318736]
 [ 21.1458004  79.0881546]
 [ 25.5940947  85.1375645]
 [ 22.7195687  75.8577258]
 [ 19.2183307  72.9780897]
 [ 23.2599333  77.412615 ]
 [ 17.6868159  83.2184815]
 [ 22.3071588  73.1812187]
 [ 27.1591006  78.3957574]
 [ 30.900965   75.8572758]
 [ 22.3038945  70.8021599]
 [ 27.1766701  78.0080745]
 [ 26.7271012  88.3952861]
 [ 19.9974533  73.7898023]
 [ 28.4089123  77.3177894]
 [ 30.3397809  76.3868797]
 [ 28.9844618  77.7064137]
 [ 19.2405315  73.1282632]
 [ 19.3919275  72.8397317]
 [ 25.3176452  82.9739144]
 [ 34.0836708  74.7972825]
 [ 23.7956531  86.4303859]
 [ 26.2389469  73.0243094]
 [ 31.6339793  74.8722642]
 [ 21.2513844  81.6296413]
 [ 25.4358011  81.846311 ]
 

In [19]:
aq = rbfi(cities[:, 0], cities[:, 1])
aq = map(lambda x: x if x > 0.0 else 0.0, aq)
print aq

[40.375000000002785, 182.68299999999545, 74.806666666667866, 183.34999999999741, 61.663333333330911, 70.123333333332766, 121.16560406774283, 64.140000000003511, 103.9300000000012, 100.45430890166827, 164.4333333333343, 166.36000000000513, 84.389999999998963, 148.85000000000264, 125.51277153301044, 64.260000000006585, 139.3094807938387, 45.880000000001701, 130.41542105728934, 91.476546105905982, 0.0, 135.64085575413969, 57.808440642437745, 81.412998403242455, 68.630000000008977, 161.04000000000087, 19.871714928747679, 326.35931087244899, 75.998522497566569, 70.696098289690326, 151.13000000000056, 0.0, 187.19785446541255, 95.800000000005184, 0.0, 96.767032604892847, 149.19280407746601, 85.759472860765413, 127.04503998208793, 77.60040452598696, 42.855353984331487, 83.217252753750586, 62.35823275832422, 33.531877575256658, 0.0, 357.33115707251181, 345.18738851705893, 124.34000000000378, 150.1950185297037, 38.830000000003906, 211.86689779321705, 0.0, 93.589759839066119, 111.56589994615365, 

In [20]:
fig = gmaps.figure(center=india_coordinates, zoom_level=4)

info_box_template = """
<div>
<p><b>City:</b> {0}</p>
<p><b>PM2.5:</b> {1:.2f}</p>
</div>
"""
city_info = [info_box_template.format(data[i], aq[i]) for i in range(0, len(data))]

marker_layer = gmaps.marker_layer(cities, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

A Jupyter Widget

In [21]:
embed_minimal_html('national-aq-interp.html', views=[fig])

In [22]:
delhi_coordinates = decode_address_to_coordinates("New Delhi").values()

In [7]:
f = open('data/Delhi_Delhi.txt', 'r')
delhi_data = f.readlines()
delhi_data = filter(lambda elem: elem != "\n", delhi_data)
delhi_data = map(lambda elem: elem.split(","), delhi_data)
filtered_delhi_data = []
station = -1
for line in delhi_data:
    if len(line) > 2 and "station" in line[2]:
        station = line[2].split(":")[1].rstrip("\n")
    if line[0] == "2016":
        filtered_delhi_data.append([station, line[1].rstrip("\n")])

delhi_data = filtered_delhi_data

f = open('id-mappings/station-ids.txt', 'r')
stations = f.readlines()
stations = map(lambda elem: elem.split(","), stations)
stations = filter(lambda elem: elem[2] == "85" and elem[4].isdigit(), stations)
stations = {station[4]:station[5].rstrip("\n") for station in stations}
print stations, delhi_data

{'6': 'Dwarka', '58': 'Noida Sector 62', '46': 'Mandir Marg', '54': 'Anand Vihar', '45': 'R K Puram', '51': 'Gole Market', '53': 'Civil Lines', '52': 'IGI Airport', '1': 'MOBILE VAN', '55': 'East Arjun Nagar-Delhi CPCB', '3': 'DTU', '2': 'ITO', '5': 'Shadipur', '4': 'Sirifort', '7': 'IHBAS', '47': 'Punjabi Bagh'} [['5', '144.68'], ['3', '326.36'], ['47', '139.61'], ['4', '278.77'], ['7', '145.87'], ['45', '135.92'], ['2', '216.32'], ['46', '119.59'], ['6', '143.32'], ['54', '176.39']]


In [8]:
delhi_station_coordinates = []
for station in delhi_data:
    delhi_station_coordinates.append(decode_address_to_coordinates("{}, Delhi".format(stations[station[0]])).values())
    sleep(1)
print delhi_station_coordinates

[[28.6515584, 77.15819359999999], [28.7500749, 77.11766519999999], [28.66197529999999, 77.1241557], [28.5511631, 77.2277579], [28.6809824, 77.3051949], [28.5660075, 77.1767435], [28.6308294, 77.2505733], [28.6597298, 77.2827525], [28.5921401, 77.0460481], [28.650218, 77.30270589999999]]


In [9]:
delhi_coordinates = decode_address_to_coordinates("New Delhi, Delhi").values()
fig = gmaps.figure(center=delhi_coordinates, zoom_level=11)
weights = np.array(delhi_data)[:, 1]
heatmap_layer = gmaps.heatmap_layer(delhi_station_coordinates, weights=weights)
heatmap_layer.max_intensity = 200
heatmap_layer.point_radius = 35.0

fig.add_layer(heatmap_layer)

info_box_template = """
<div>
<p><b>Station:</b> {0}</p>
<p><b>PM2.5:</b> {1}</p>
</div>
"""
print delhi_data
station_info = [info_box_template.format(stations[d[0]], d[1]) for d in delhi_data]

marker_layer = gmaps.marker_layer(delhi_station_coordinates, info_box_content=station_info)
fig.add_layer(marker_layer)

fig

[['5', '144.68'], ['3', '326.36'], ['47', '139.61'], ['4', '278.77'], ['7', '145.87'], ['45', '135.92'], ['2', '216.32'], ['46', '119.59'], ['6', '143.32'], ['54', '176.39']]


A Jupyter Widget

In [10]:
embed_minimal_html('delhi-aq-pm25.html', views=[fig])